In [1]:
from data_tools import get_full_text_data, get_data_with_dates

import pandas as pd

In [2]:
dates = get_data_with_dates(get_full_text_data())

df = pd.DataFrame(columns=['topic', 'id', 'text'])
for row in dates.itertuples():
    src_df = pd.read_excel(row.path)
    new_df = pd.DataFrame(columns=['topic', 'id', 'text'])
    new_df['id'] = src_df.index
    new_df['text'] = src_df.apply(lambda x: " ".join([*str(x['Lead']).splitlines(), *str(x['OCR']).splitlines()]) + '\n', axis=1).tolist()
    new_df['topic'] = row.name
    df = pd.concat((df, new_df), ignore_index=True)

df.to_feather('saved_objects/full_text_df.feather')
with open('saved_objects/texts.txt', 'w') as f:
    f.writelines(df['text'].to_list())

/home/proto/repos/crisis-detector/data_tools.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['path'] = data1['Nazwa pliku'].apply(lambda x: files[fnames.index(x)])


In [3]:
df['text']

0         Chciałoby się rzec, że to będzie piękny dzień ...
1         Jessica Mercedes jest znana każdemu , kto choć...
2         2192153632  Zobacz galerię  kolaż ELLE.pl32  Z...
3         14291078mat. prasoweatobiasz"Elle.plhttp://www...
4         Znaki zodiaku, układy planet, motyw gwiazdy i ...
                                ...                        
326880    CALL FOR PAPERS. Przedłużamy czas na nadsyłani...
326881    Natalka z „M jak miłość” nagle zniknęła. Teraz...
326882    Archiwum Organizatora  CO jest GRANE » 436 wyś...
326883    UWAGA, UWAGA!    Od dziś cykl publikacji "ABC ...
326884    MEWA, spektakl dyplomowy studentów Wydziału Ak...
Name: text, Length: 326885, dtype: object

In [4]:
print(df.iloc[326884]['text'])

MEWA, spektakl dyplomowy studentów Wydziału Aktorskiego Szkoły Filmowej w Łodzi, w reż. Grzegorza Wiśniewskiego, otrzymał Nagrodę Specjalną Festival Of International Student Theatre w Belgradzie. FIST Festival Of International Student Theatre odbył się w dniach 21-25 maja, w Belgradzie._x000D_ _x000D_ Nasz spektakl otrzymał Nagrodę Specjalną, którą w imieniu zespołu odebrała Mariola Kukuła._x000D_ _x000D_ Szczegóły: facebook.com/FestivalOfInternationalStudentTheatre_x000D_ _x000D_ Obejrzyj podziękowania naszych aktorek i aktorów: vimeo.com_x000D_ _x000D_ MEWA_x000D_ _x000D_ Anton Czechow_x000D_ _x000D_ Przekład: Natalia Gałczyńska_x000D_ _x000D_ Reżyseria, scenografia, kostiumy, reżyseria światła:_x000D_ _x000D_ Grzegorz Wiśniewski_x000D_ _x000D_ Obsada:_x000D_ _x000D_ BORYS TRIGORIN – Michał Darewski_x000D_ _x000D_ SIEMION MIEDWIEDIENKO / ILIA SZAMRAJEW – Sebastian Dela_x000D_ _x000D_ MASZA – Klaudia Janas_x000D_ _x000D_ PAULINA – Karolina Kostoń_x000D_ _x000D_ NINA ZARIECZNA – Justy